In [ ]:
import folium
import DownloadData as DD
import DataSorting as DS
import json
import geojson
from urllib.request import urlopen
from geojson_utils import centroid

In [ ]:
with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/regions.geojson') as f:
    regions = geojson.load(f)
with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson') as f:
    departements = geojson.load(f)
with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/cantons.geojson') as f:
    cantons = geojson.load(f)

In [ ]:
# récupération des données sur les stations-service
DD.get_ten_Mins_Data()
DD.get_dailyData()

# on récupère le J-5 pour observer la variation 
chaine = DD.get_chaine()

# calcul des prix moyen pour les régions/départements/cantons
reg_prices = DS.DictRegions("PrixCarburants_instantane.xml")
reg_prices_B = DS.DictRegions("PrixCarburants_quotidien_" + chaine + ".xml")
dep_prices = DS.DictDepartements("PrixCarburants_instantane.xml")
dep_prices_B = DS.DictDepartements("PrixCarburants_quotidien_" + chaine + ".xml")
can_prices = DS.DictCantons("PrixCarburants_instantane.xml")
can_prices_B = DS.DictCantons("PrixCarburants_quotidien_" + chaine + ".xml")

# calcul des variations
variations_reg = {key: reg_prices[key] - reg_prices_B.get(key, 0) for key in reg_prices.keys()}
variations_dep = {key: dep_prices[key] - dep_prices_B.get(key, 0) for key in dep_prices.keys()}
variations_can = {key: can_prices[key] - can_prices_B.get(key, 0) for key in can_prices.keys()}

In [ ]:
# tendance régionale
average_reg = folium.Map(location=[46.232192999999995, 2.209666999999996], zoom_start=5)

folium.Choropleth(
    geo_data=regions,
    name="choropleth",
    data=reg_prices,
    key_on="feature.properties.nom",
    fill_color="YlOrRd",
    nan_fill_color="White", #Use white color if there is no data available for the departements
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Diesel prices in France regions", #title of the legend
    highlight=True,
    line_color='black'
).add_to(average_reg)

folium.LayerControl().add_to(average_reg)

for i in range (0, len(regions["features"])):
    if regions["features"][i].geometry.type == 'Polygon':
        box_str = str(regions["features"][i].geometry)
        box = json.loads(box_str)
        center = centroid(box)
        nom_r = regions["features"][i]["properties"]["nom"]
        if nom_r in variations_reg:
            if variations_reg[nom_r] < 0:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_r}</br>{round(variations_reg[nom_r], 4)}',
                                  icon = folium.features.CustomIcon('down.png',icon_size=(12,12))).add_to(average_reg)
            elif 0 < variations_reg[nom_r] <= max(variations_reg.values())/1.5:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_r}</br>+{round(variations_reg[nom_r], 4)}',
                                  icon = folium.features.CustomIcon('up_1.png',icon_size=(12,12))).add_to(average_reg)
            elif max(variations_reg.values())/1.5 < variations_reg[nom_r] <= max(variations_reg.values()):
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_r}</br>+{round(variations_reg[nom_r], 4)}',
                                  icon = folium.features.CustomIcon('up_2.png',icon_size=(12,12))).add_to(average_reg)
            else:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = nom_r,
                                  icon = folium.features.CustomIcon('equal.png',icon_size=(16,16))).add_to(average_reg)
                
average_reg

In [ ]:
# tendance départementale
average_dep = folium.Map(location=[46.232192999999995, 2.209666999999996], zoom_start=5)

folium.Choropleth(
    geo_data=departements,
    name="choropleth",
    data=dep_prices,
    key_on="feature.properties.code",
    fill_color="YlOrRd",
    nan_fill_color="White", #Use white color if there is no data available for the departements
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Diesel prices in France departements", #title of the legend
    highlight=True,
    line_color='black'
).add_to(average_dep)

folium.LayerControl().add_to(average_dep)

for i in range (0, len(departements["features"])):
    if departements["features"][i].geometry.type == 'Polygon':
        box_str = str(departements["features"][i].geometry)
        box = json.loads(box_str)
        center = centroid(box)
        code_p = departements["features"][i]["properties"]["code"]
        if code_p in variations_dep:
            if variations_dep[code_p] < 0:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'CP:{code_p}</br>{round(variations_dep[code_p], 4)}',
                                  icon = folium.features.CustomIcon('down.png',icon_size=(12,12))).add_to(average_dep)
            elif 0 < variations_dep[code_p] <= max(variations_dep.values())/1.5:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'CP:{code_p}</br>+{round(variations_dep[code_p], 4)}',
                                  icon = folium.features.CustomIcon('up_1.png',icon_size=(12,12))).add_to(average_dep)
            elif max(variations_dep.values())/1.5 < variations_dep[code_p] <= max(variations_dep.values()):
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'CP:{code_p}</br>+{round(variations_dep[code_p], 4)}',
                                  icon = folium.features.CustomIcon('up_2.png',icon_size=(12,12))).add_to(average_dep)
            else:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'CP:{code_p}',
                                  icon = folium.features.CustomIcon('equal.png',icon_size=(16,16))).add_to(average_dep)
                
average_dep

In [ ]:
# tendance cantonale
average_can = folium.Map(location=[46.232192999999995, 2.209666999999996], zoom_start=5)

folium.Choropleth(
    geo_data=cantons,
    name="choropleth",
    data=can_prices,
    key_on="feature.properties.nom",
    fill_color="YlOrRd",
    nan_fill_color="White", #Use white color if there is no data available for the departements
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Diesel prices in France cantons", #title of the legend
    highlight=True,
    line_color='black'
).add_to(average_can)

folium.LayerControl().add_to(average_can)

for i in range (0, len(cantons["features"])):
    if cantons["features"][i].geometry.type == 'Polygon':
        box_str = str(cantons["features"][i].geometry)
        box = json.loads(box_str)
        center = centroid(box)
        nom_c = cantons["features"][i]["properties"]["nom"]
        if nom_c in variations_can:
            if variations_can[nom_c] < 0:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_c}</br>{round(variations_can[nom_c], 4)}',
                                  icon = folium.features.CustomIcon('down.png',icon_size=(12,12))).add_to(average_can)
            elif 0 < variations_can[nom_c] <= max(variations_can.values())/1.5:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_c}</br>+{round(variations_can[nom_c], 4)}',
                                  icon = folium.features.CustomIcon('up_1.png',icon_size=(12,12))).add_to(average_can)
            elif max(variations_can.values())/1.5 < variations_can[nom_c] <= max(variations_can.values()):
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = f'{nom_c}</br>+{round(variations_can[nom_c], 4)}',
                                  icon = folium.features.CustomIcon('up_2.png',icon_size=(12,12))).add_to(average_can)
            else:
                folium.Marker(location = [center['coordinates'][1], center['coordinates'][0]],
                                  popup = nom_c,
                                  icon = folium.features.CustomIcon('equal.png',icon_size=(16,16))).add_to(average_can)
                
average_can